In [1]:
# switch to the project directory
%cd ..
# working directory should be ../FSE

C:\Users\admin\Desktop\research\pdi


In [2]:
import sys
import os

module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pdi.constants import (
    NUM_WORKERS, )

from pdi.data.constants import N_COLUMNS
from pdi.data.types import Split
import wandb
from pdi.train import train


In [4]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
device = torch.device("cuda")

In [5]:
from math import sqrt
target_code = 321



def main(train_loader, val_loader, create_model_fun):
    wandb.init(allow_val_change=True)
    if "d_model" in wandb.config:
        wandb.config.start_lr = 1e-3 / sqrt(wandb.config.d_model)
    pos_weight = torch.tensor(1.0).float().to(device)
    wandb.log({"pos_weight": pos_weight.item()})
    model = create_model_fun()
    train(model, target_code, device, train_loader, val_loader, pos_weight)

Delete Sweep

In [ ]:
delete_sweep_configuration = {
    "method": "random",
    "metric": {
        "goal": "maximize",
        "name": "val_f1"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 10,
        "eta": 2,
    },
    "parameters": {
        "bs": {
            "value": 256
        },
        "max_epochs": {
            "value": 100
        },
        "start_lr": {
            "value": 5e-3
        },
        "patience": {
            "value": 10
        },
        "patience_threshold": {
            "value": 0.001
        },
        "h0": {
            "values": [16, 32, 64, 128, 256]
        },
        "h_layers": {
            "values": [1, 2, 3]
        },
        "h_scale": {
            "values": [0.125, 0.25, 0.5, 0.75]
        },
        "dropout": {
            "value": 0.1
        },
        "gamma": {
            "value": 0.9
        }
    },
}

In [ ]:
from pdi.models import NeuralNet


def create_simple_model():
    hidden_layers = [wandb.config.h0]
    for i in range(wandb.config.h_layers - 1):
        hidden_layers.append(int(wandb.config.h_scale * hidden_layers[-1]))

    return NeuralNet(
        [N_COLUMNS, *hidden_layers, 1],
        nn.ReLU,
        wandb.config.dropout,
    ).to(device)


In [ ]:
from pdi.data.preparation import DeletePreparation

sweep_id = wandb.sweep(sweep=delete_sweep_configuration, project="DeleteSweep")

bs = delete_sweep_configuration["parameters"]["bs"]["value"]
train_loader, val_loader = DeletePreparation().prepare_dataloaders(
       bs, NUM_WORKERS, [Split.TRAIN, Split.VAL])
wandb.agent(sweep_id, function=lambda: main(train_loader, val_loader, create_simple_model))

FSE Sweep

In [6]:
fse_sweep_configuration = {
    "method": "random",
    "metric": {
        "goal": "minimize",
        "name": "val_loss"
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 4,
        "eta": 2,
    },
    "parameters": {
        "bs": {
            "value": 512
        },
        "max_epochs": {
            "value": 40
        },
        "patience": {
            "value": 5
        },
        "patience_threshold": {
            "value": 0.001
        },
        "dropout": {
            "value": 0.1
        },
        "gamma": {
            "value": 0.9
        },
        "embed_hidden": {
            "values": [16, 32, 64, 128]
        },
        "d_model": {
            "values": [16, 32, 64]
        },
        "ff_hidden": {
            "values": [16, 32, 64, 128, 256]
        },
        "pool_hidden": {
            "values": [16, 32, 64, 128]
        },
        "num_heads": {
            "values": [1, 2, 4, 8]
        },
        "num_blocks": {
            "values": [1, 2, 4, 6]
        },
    },
}

In [7]:
from pdi.models import AttentionModel


def create_attention_model():
    return AttentionModel(
        N_COLUMNS + 1,
        wandb.config.embed_hidden,
        wandb.config.d_model,
        wandb.config.ff_hidden,
        wandb.config.pool_hidden,
        wandb.config.num_heads,
        wandb.config.num_blocks,
        nn.ReLU,
        wandb.config.dropout,
    ).to(device)


In [8]:
from pdi.data.preparation import FeatureSetPreparation

sweep_id = wandb.sweep(sweep=fse_sweep_configuration, project="FSESweep")

bs = fse_sweep_configuration["parameters"]["bs"]["value"]

train_loader, val_loader = FeatureSetPreparation().prepare_dataloaders(
        bs, NUM_WORKERS, [Split.TRAIN, Split.VAL])
wandb.agent(sweep_id, function=lambda: main(train_loader, val_loader, create_attention_model))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: mhvufvgq
Sweep URL: https://wandb.ai/mkasak/FSESweep/sweeps/mhvufvgq


wandb: Agent Starting Run: l44ynx2e with config:
wandb: 	bs: 512
wandb: 	d_model: 32
wandb: 	dropout: 0.1
wandb: 	embed_hidden: 128
wandb: 	ff_hidden: 128
wandb: 	gamma: 0.9
wandb: 	max_epochs: 40
wandb: 	num_blocks: 4
wandb: 	num_heads: 8
wandb: 	patience: 5
wandb: 	patience_threshold: 0.001
wandb: 	pool_hidden: 64
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mkasak. Use `wandb login --relogin` to force relogin


100%|████████████████████████████████████████████████████████████████████████████████| 807/807 [00:15<00:00, 53.62it/s]


Epoch: 0, F1: 0.5367


...

Epoch: 24, F1: 0.8545


100%|████████████████████████████████████████████████████████████████████████████████| 807/807 [00:13<00:00, 59.88it/s]


Epoch: 25, F1: 0.8549
Finishing training early at epoch: 25


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▄▃▃▂▂▂▂▂▁▂▂▁▂▂▂▂▂▁▁▂▁▂▂▂▂▂▁▂▁▁▂▂▁▂▂▁▂▁▂
pos_weight,▁
scheduled_lr,█▇▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
val_f1,▁▆▇▇██████████████████████
val_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▁▅▅▆▇▇▇▇▇▇▇▇▆▇▇▇███▇███▇██
val_recall,▁▆▇██▇██████████▇▇██▇███▇▇
val_threshold,▁▂▁▂▄▆▅▆▇▆▆▆▃▆▅▆▇▇▇▆▇▇▇▅▇█
epoch,25
loss,0.74403


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
